# Test poke-env api

You need to install poke-env api by
```{commandline}
$ pip install poke-env
```
The poke-env requirements does not include dataclasses module, so if error occurs importing poke_env, you need to install it manually.

```{commandline}
$ pip install dataclasses
```

Once that is complete, clone the pokemon-showdown implementation
```{commandline}
$ git clone https://github.com/hsahovic/Pokemon-Showdown.git
```


### Configuring Players

Create random players.
Before running the below code, you should be running your pokemon-showdown server on your localhost.

In [ ]:
import asyncio

from poke_env.player.random_player import RandomPlayer
from poke_env.player.utils import cross_evaluate
from tabulate import tabulate

Create 3 RandomPlayer agents and battle with each other 20 times each.

In [ ]:
players = [RandomPlayer(max_concurrent_battles=10, battle_format="gen4randombattle") for _ in range(3)]

In [ ]:
cross_evaluation = await cross_evaluate(players, n_challenges=20)

table = [["-"] + [p.username for p in players]]

for p_1, results in cross_evaluation.items():
    table.append([p_1] + [cross_evaluation[p_1][p_2] for p_2 in results])

print(tabulate(table))

### Max-damage player (Heuristic)

In [ ]:
from poke_env.player.player import Player
from poke_env.environment.battle import Battle

Create a max damage player that chooses a move with maximum damage

In [ ]:
class MaxDamagePlayer(Player):
    def choose_move(self, battle: Battle) -> str:
        # If the player can attack, it will
        if battle.available_moves:
            # Finds the best move among available ones
            best_move = max(battle.available_moves, key=lambda move: move.base_power)
            return self.create_order(best_move)
        # If no attack is available, a random switch will be made
        else:
            return self.choose_random_move(battle)

Cross evaluate with RandomPlayer

In [ ]:
random_player = RandomPlayer(battle_format="gen4randombattle")
max_damage_player = MaxDamagePlayer(battle_format="gen4randombattle")

In [ ]:
await max_damage_player.battle_against(random_player, n_battles=100)

print(f"Max damage player won {max_damage_player.n_won_battles} out of 100 battles")

### Battle against max damage player

Go to your local server of pokemon showdown. Log in, and challenge to the player. You can find the player by the below name.

In [ ]:
print(max_damage_player.username)

After you challenge with gen4randombattle, run the below code.

In [ ]:
await max_damage_player.accept_challenges(None, 1)